# 5Andar

## Abordagem usando os dados para chat de LLM com api OPENAI

In [5]:
import sys

from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
import os

In [6]:
os.environ['OPENAI_API_KEY'] = 'KEY'

É nescessario comstruir um template para explicar para o Chat o que significada cada coluna no csv.

In [27]:
PREFIX = '''You are an AI data analist. 

'''

FORMAT_INSTRUCTIONS = """
Com base nos 3 dataframes encontre a resposata da pergunta do usuario usando os dados e o dicionario de dados.

Using the data dictionary:
'''
Df1: Tabela de planos de mídia (Esta tabela contém, pordia, o custo das campanhas em cada uma das cidades.)
dt_insertion: Data em que a campanha foi para o ar.
city_group: Cidade em que a campanha foi efetuada.
media: Canal em que foi veiculado a campanha.
daily_cost: Investimento em reais das ações de marketing na data em questão.

Df2: Tabela de leads gerados (Esta tabela contém, por dia, a quantidade de proprietários cadastrados (leads) em cada uma das cidades.)
dt_event: Data de referência em que os leads foram captados.
city_group: Cidade em que o lead foi captado
leads_5a: Total de proprietários cadastrados cujos imóveis estão dentro das nossas áreas de operação.
discards: Leads que chegam no 5A, mas são descartados por algum motivo.
qualifields: Leads qualificados, passam em todos os critérios do 5A.


Df3: Tabela de área de operações (Nesta tabela consta a quantidade de bairros com e sem operações, leads totais, descartados e aproveitados pelo QuintoAndar (5A).)
nbr_bairros_abertos: Quantidade de bairros que o 5A atua.
nbr_bairros_sem_ops: Quantidade de bairros que o 5A não possui operação.
leads: Total de proprietários que se cadastraram no nosso produto.
leads_nops: Proprietários cadastrados que foram descartados por seus imóveis estarem fora da área de operação do 5A.
contratos_assinados: Quantidade de contratos de aluguéis assinados.
tenant_prospects: Quantidade de inquilinos que passaram pelo funil de demanda.
tenant_prospect_to_contract_signed: A razão entre quantidade de contratos assinados e quantidade de inquilinos que passaram pelo funil de demanda.
ticket_medio: Valor médio dos contratos 5A de aluguéis da região.
'''

"""

SUFFIX = '''

Begin!

Previous conversation history:
{chat_history}

Instructions: {input}

'''

agent = create_csv_agent(
    #ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo-0613"),
    ChatOpenAI(temperature=0.5, model="gpt-4-1106-preview"),
    
    [r"C:\Users\iulih\Meu Drive\Work\PS\QuintoAndar\data_csv\leads.csv", r"C:\Users\iulih\Meu Drive\Work\PS\QuintoAndar\data_csv\media_plan.csv",r'C:\Users\iulih\Meu Drive\Work\PS\QuintoAndar\data_csv\operantions.csv'],
    #memory=memory,use_memory=True,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={
        'prefix': PREFIX, 
        'format_instructions': FORMAT_INSTRUCTIONS,
        'suffix': SUFFIX
    }
)

#Primeira pergunta:

In [31]:
agent.run('Olhando para qualifieds, custo, tenant_prospect_to_contract_signed, ticket_medio e o que você achar melhor, Identifique os locais em que as campanhas foram mais efetivas e qual cidade foi menos efetiva')



> Entering new AgentExecutor chain...
Thought: To answer this question, I will need to combine information from the three dataframes. I will look at the 'qualifieds' column from df1, the 'daily_cost' column from df2, and the 'tenant_prospect_to_contract_signed' and 'ticket_medio' columns from df3. Since the city names are in Portuguese, I will assume that 'city_group' refers to the same cities across the dataframes, even though the events and insertion orders might not be directly linked by date. I will need to calculate the effectiveness of the campaigns, which could involve a ratio of qualified leads to cost and also consider the conversion rate to contracts signed and average ticket size. The city with the highest ratio might be considered the most effective, while the one with the lowest ratio could be considered the least effective.

Action: I will begin by merging the dataframes on the 'city_group' column to create a combined dataframe. I will then calculate the effectiveness o

'The most effective campaign was in Belo Horizonte, and the least effective campaign was in Rio de Janeiro.'

No log é possivel entender cada passo a passo feito pela OpenAI. É possivel usar outros modelos até mesmo OFFLine o que depedenria de um processamento alto em GPU.